# 03 - Camada Ouro: Dados Prontos para Consumo e Análise

In [1]:
import pandas as pd 
import os 
import shutil
import glob

In [2]:
df = pd.read_parquet("../notebooks/data/silver/breweries.parquet/", engine="pyarrow")
print(df.head())

                                     ID  \
0  0faa0fb2-fffa-416d-9eab-46f67477c8ef   
1  4f4b5b34-d572-4dff-a18f-47e507c073e6   
2  ef970757-fe42-416f-931d-722451f1f59c   
3  5ae467af-66dc-4d7f-8839-44228f89b596   
4  4788221a-a03b-458c-9084-4cadd69ade6d   

                                            NAME TYPE BREWERY  \
0  12 West Brewing Company - Production Facility        MICRO   
1                                   1912 Brewing        MICRO   
2                           10 Barrel Brewing Co        LARGE   
3                      101 North Brewing Company       CLOSED   
4                     14 Cannons Brewing Company        MICRO   

               CITY POSTAL CODE        COUNTRY   LONGITUDE   LATITUDE  \
0              MESA       85207  UNITED STATES -111.586066  33.436188   
1            TUCSON  85745-1444  UNITED STATES -110.992750  32.246737   
2         SAN DIEGO  92101-6618  UNITED STATES -117.129593  32.714813   
3          PETALUMA  94954-7100  UNITED STATES -122.665055

In [3]:
df_state_type = (
    df.groupby(['STATE PROVINCE', 'TYPE BREWERY'],observed=True)
      .size()
      .reset_index(name='COUNT_BREWERY')
      .sort_values(by='COUNT_BREWERY',ascending=False)
)
display(df_state_type)

,STATE PROVINCE,TYPE BREWERY,COUNT_BREWERY
25,OREGON,LARGE,4
11,INDIANA,MICRO,3
0,ARIZONA,MICRO,2
24,OREGON,BREWPUB,2
30,WISCONSIN,MICRO,2
3,CALIFORNIA,MICRO,2
5,COLORADO,LARGE,1
6,COLORADO,MICRO,1
29,WASHINGTON,MICRO,1
28,WASHINGTON,CLOSED,1


In [4]:
df_country = (
             df.groupby(['COUNTRY'], observed=True)
               .size()
               .reset_index(name='COUNT_COUNTY')
               .sort_values(by='COUNT_COUNTY',ascending=False)
)
display(df_country)

,COUNTRY,COUNT_COUNTY
1,UNITED STATES,39
0,IRELAND,1


In [5]:
df_type = (
    df.groupby(['TYPE BREWERY'], observed=True)
      .size()
      .reset_index(name='COUNT_TYPE')
      .sort_values(by='COUNT_TYPE', ascending=False)
)

display(df_type)


,TYPE BREWERY,COUNT_TYPE
3,MICRO,24
2,LARGE,7
0,BREWPUB,6
1,CLOSED,2
4,PROPRIETOR,1


In [6]:
df_agg_estado = (
    df.groupby(['STATE PROVINCE'],observed=True)
      .size()
      .reset_index(name='TOTAL BREWERIES')
      .sort_values(by='TOTAL BREWERIES', ascending=False)
)
display(df_agg_estado)


,STATE PROVINCE,TOTAL BREWERIES
18,OREGON,6
1,CALIFORNIA,4
2,COLORADO,4
6,INDIANA,3
0,ARIZONA,2
21,WASHINGTON,2
16,OHIO,2
22,WISCONSIN,2
8,LAOIS,1
9,MASSACHUSETTS,1


In [7]:
top_states = df_agg_estado.head(10)
display(top_states)


,STATE PROVINCE,TOTAL BREWERIES
18,OREGON,6
1,CALIFORNIA,4
2,COLORADO,4
6,INDIANA,3
0,ARIZONA,2
21,WASHINGTON,2
16,OHIO,2
22,WISCONSIN,2
8,LAOIS,1
9,MASSACHUSETTS,1


In [8]:
df_sem_site = (
    df[~df['URL WEBSITE'].str.contains(r'\.', na=False)]
      .sort_values(by=['STATE PROVINCE', 'NAME'], ascending=[True, True])
)
print("*-----------------------------------------------*")
print(f"Cervejarias SEM site: {len(df_sem_site)} de {len(df)} ({round(100 * len(df_sem_site) / len(df), 2)}%)")
print("*-----------------------------------------------*")
display(df_sem_site)


*-----------------------------------------------*
Cervejarias SEM site: 8 de 40 (20.0%)
*-----------------------------------------------*


,ID,NAME,TYPE BREWERY,CITY,POSTAL CODE,COUNTRY,LONGITUDE,LATITUDE,PHONE,URL WEBSITE,STATE PROVINCE
0,0faa0fb2-fffa-416d-9eab-46f67477c8ef,12 West Brewing Company - Production Facility,MICRO,MESA,85207,UNITED STATES,-111.586066,33.436188,0,Unknown,ARIZONA
6,1988eb86-f0a2-4674-ba04-02454efa0d31,10 Barrel Brewing Co - Denver,LARGE,DENVER,80205-2231,UNITED STATES,-104.985366,39.759251,7205738992,Unknown,COLORADO
13,7531dbd8-afc9-4b5b-95bc-7ece7f2c0bf3,10-56 Brewing Company,MICRO,KNOX,46534,UNITED STATES,-86.627954,41.289715,6308165790,Unknown,INDIANA
15,add7f978-942e-4d56-b209-c80837a51d69,18th Street Brewery,MICRO,HAMMOND,46320-1817,UNITED STATES,-87.517422,41.615568,0,Unknown,INDIANA
19,d5cb896d-3e99-4e19-9693-5e06ce987e53,127 Brewing,MICRO,JACKSON,49201-7010,UNITED STATES,-84.431168,42.286672,5172581346,Unknown,MICHIGAN
21,46839a79-b7bf-4733-b91b-ce116d062a57,1817 Brewery,MICRO,OKOLONA,38860,UNITED STATES,-88.750264,34.001703,6623055907,Unknown,MISSISSIPPI
29,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,10 Barrel Brewing Co,LARGE,BEND,97703-2465,UNITED STATES,-121.328802,44.057565,5415851007,Unknown,OREGON
31,9f1852da-c312-42da-9a31-097bac81c4c0,10 Barrel Brewing Co - Bend Pub,LARGE,BEND,97701,UNITED STATES,-121.280954,44.091211,5415851007,Unknown,OREGON


In [9]:
gold_path = "../data/gold/breweries.parquet"

In [10]:
if os.path.exists(gold_path):
    files = glob.glob(os.path.join(gold_path, "*"))
    for f in files:
        if os.path.isdir(f):
            shutil.rmtree(f)
        else:
            os.remove(f)

In [12]:
df_state_type.to_parquet('../notebooks/data/gold/df_state_type.parquet', index=False)
df_country.to_parquet('../notebooks/data/gold/df_country.parquet', index=False)
df_type.to_parquet('../notebooks/data/gold/df_type.parquet', index=False)
df_agg_estado.to_parquet('../notebooks/data/gold/df_agg_estado.parquet', index=False)
top_states.to_parquet('../notebooks/data/gold/top_states.parquet', index=False)
df_sem_site.to_parquet('../notebooks/data/gold/df_sem_site.parquet', index=False)